# **Oficina - Automação de Relatórios com Python**

Oficina para explicar como utilizar prompt do ChatGPT para criar um código python para automatizar relatórios.

## **1. Adição de Texto em um Template Word**

**Pedido 1:** Chat, eu gostaria de criar um código python para preencher uma parte de um arquivo word. Para isso, eu já tenho um template chamado de "template_relatorio.docx". Eu gostaria de adicionar um texto nesse documento na parte onde tem <TEXTO>, como fazer?


In [1]:
pip install python-docx

Note: you may need to restart the kernel to use updated packages.


In [2]:
from docx import Document

In [28]:
def preencher_documento(caminho_template, marcador, texto_substituto, caminho_saida):
    # Carrega o documento template
    doc = Document(caminho_template)
    
    # Itera sobre os parágrafos do documento
    for paragrafo in doc.paragraphs:
        if marcador in paragrafo.text:
            # Substitui o marcador pelo texto desejado
            paragrafo.text = paragrafo.text.replace(marcador, texto_substituto)
    
    # Salva o documento modificado com um novo nome
    doc.save(caminho_saida)

In [29]:
# Caminho para o template e onde o documento preenchido será salvo
caminho_template = 'template_relatorio.docx'
caminho_saida = 'relatorio_preenchido.docx'

# Texto para substituir o marcador
texto_substituto = 'Este é o texto substituto'

# Chama a função para preencher o documento
preencher_documento(caminho_template, '<TEXTO>', texto_substituto, caminho_saida)

## **2. Adição de Planilha em um Template Word**

**Pedido 2:** Chat, eu gostaria de criar um código python para ler uma planilha de informações chamado de "dados_exemplo.csv". Em seguida adicionar as informações em formato de tabela no arquivo word chamado de "template_relatorio.docx". Eu gostaria que essa tabela fosse substituída no documento na parte onde tem <TABELA>, como fazer?

In [30]:
pip install python-docx pandas

In [37]:
import pandas as pd
from docx import Document
from docx.oxml.ns import qn

In [38]:
def criar_tabela(doc, dados):
    # Adiciona uma tabela ao documento
    tabela = doc.add_table(rows=1, cols=len(dados.columns))
    tabela.style = 'Table Grid'  # Adiciona um estilo à tabela
    
    # Adiciona cabeçalhos da tabela
    hdr_cells = tabela.rows[0].cells
    for i, coluna in enumerate(dados.columns):
        hdr_cells[i].text = coluna
    
    # Adiciona dados à tabela
    for index, linha in dados.iterrows():
        row_cells = tabela.add_row().cells
        for i, valor in enumerate(linha):
            row_cells[i].text = str(valor)
    
    return tabela

In [39]:
def substituir_tabela_no_documento(doc, marcador, dados):
    # Itera sobre os parágrafos para encontrar o marcador
    for paragrafo in doc.paragraphs:
        if marcador in paragrafo.text:
            # Remove o parágrafo com o marcador
            p = paragrafo._element
            p.getparent().remove(p)
            # Adiciona uma nova tabela com os dados
            doc.add_paragraph()  # Adiciona um parágrafo em branco antes da tabela
            tabela = criar_tabela(doc, dados)
            break

In [40]:
def atualizar_documento(caminho_csv, caminho_template, caminho_saida):
    # Lê os dados do CSV
    dados = pd.read_csv(caminho_csv)
    
    # Carrega o documento template
    doc = Document(caminho_template)
    
    # Substitui o marcador <TABELA> pela nova tabela
    substituir_tabela_no_documento(doc, '<TABELA>', dados)
    
    # Salva o documento modificado com um novo nome
    doc.save(caminho_saida)

In [41]:
# Caminhos dos arquivos
caminho_csv = 'dados_exemplo.csv'
caminho_template = 'template_relatorio.docx'
caminho_saida = 'relatorio_atualizado.docx'

# Atualiza o documento com a tabela
atualizar_documento(caminho_csv, caminho_template, caminho_saida)

## **3. Adição de Gráfico em um Template Word**

**Pedido 3:** Chat, eu tenho uma planilha chamada de "dados_exemplo.csv" que possui quatro colunas: Nome, Idade, Departamento e Salario. Eu gostaria de criar um código python para ler essas informações e criar um gráfico mostrando os salários dos nomes cadastrados. Em seguida, eu gostaria de adicionar esse gráfico em um documento word chamado de "template_relatorio.docx". Eu gostaria que o gráfico fosse substituído no documento na parte onde tem <GRAFICO>.

In [42]:
pip install pandas matplotlib python-docx

In [43]:
import pandas as pd
import matplotlib.pyplot as plt
from docx import Document
from docx.shared import Inches
import io

In [44]:
def criar_grafico(dados):
    # Cria um gráfico de barras para os salários
    plt.figure(figsize=(10, 6))
    plt.bar(dados['Nome'], dados['Salario'], color='skyblue')
    plt.xlabel('Nome')
    plt.ylabel('Salário')
    plt.title('Salário dos Funcionários')
    plt.xticks(rotation=45, ha='right')

    # Salva o gráfico em um buffer de memória
    buffer = io.BytesIO()
    plt.savefig(buffer, format='png')
    plt.close()
    buffer.seek(0)
    return buffer

In [45]:
def substituir_grafico_no_documento(doc, marcador, buffer):
    # Itera sobre os parágrafos para encontrar o marcador
    for paragrafo in doc.paragraphs:
        if marcador in paragrafo.text:
            # Remove o parágrafo com o marcador
            p = paragrafo._element
            p.getparent().remove(p)
            # Adiciona o gráfico ao documento
            doc.add_paragraph()  # Adiciona um parágrafo em branco antes do gráfico
            doc.add_picture(buffer, width=Inches(6))  # Ajuste o tamanho conforme necessário
            break

In [46]:
def atualizar_documento(caminho_csv, caminho_template, caminho_saida):
    # Lê os dados do CSV
    dados = pd.read_csv(caminho_csv)
    
    # Cria o gráfico
    buffer = criar_grafico(dados)
    
    # Carrega o documento template
    doc = Document(caminho_template)
    
    # Substitui o marcador <GRAFICO> pelo gráfico
    substituir_grafico_no_documento(doc, '<GRAFICO>', buffer)
    
    # Salva o documento modificado com um novo nome
    doc.save(caminho_saida)

In [47]:
# Caminhos dos arquivos
caminho_csv = 'dados_exemplo.csv'
caminho_template = 'template_relatorio.docx'
caminho_saida = 'relatorio_atualizado.docx'

# Atualiza o documento com o gráfico
atualizar_documento(caminho_csv, caminho_template, caminho_saida)

## **4. Combinação dos Códigos em um Único Script**

**Pedido 4:** Chat, agora eu quero que você combine os pedidos 1, 2 e 3 em apenas um único script. Separe as importações, os métodos e a execução. O script deve receber os seguintes parâmetros (i) o documento "template_relatorio.docx" para adicionar as informações; (ii) o texto que vai ser inserido no documento na parte <TEXTO> do documento; (iii) "dados exemplo.csv" com as informações da tabela para serem inseridas na parte <TABELA> do documento; e (iv) a geração do gráfico para ser inserido na parte de <GRAFICO> do documento.

In [48]:
import pandas as pd
import matplotlib.pyplot as plt
from docx import Document
from docx.shared import Inches
import io

In [49]:
# Funções

def criar_tabela(doc, dados):
    tabela = doc.add_table(rows=1, cols=len(dados.columns))
    tabela.style = 'Table Grid'
    
    hdr_cells = tabela.rows[0].cells
    for i, coluna in enumerate(dados.columns):
        hdr_cells[i].text = coluna
    
    for _, linha in dados.iterrows():
        row_cells = tabela.add_row().cells
        for i, valor in enumerate(linha):
            row_cells[i].text = str(valor)
    
    return tabela

In [50]:
def criar_grafico(dados):
    plt.figure(figsize=(10, 6))
    plt.bar(dados['Nome'], dados['Salario'], color='skyblue')
    plt.xlabel('Nome')
    plt.ylabel('Salário')
    plt.title('Salário dos Funcionários')
    plt.xticks(rotation=45, ha='right')

    buffer = io.BytesIO()
    plt.savefig(buffer, format='png')
    plt.close()
    buffer.seek(0)
    return buffer

In [51]:
def substituir_texto_no_documento(doc, marcador, texto_substituto):
    for paragrafo in doc.paragraphs:
        if marcador in paragrafo.text:
            paragrafo.text = paragrafo.text.replace(marcador, texto_substituto)

In [52]:
def substituir_tabela_no_documento(doc, marcador, dados):
    for paragrafo in doc.paragraphs:
        if marcador in paragrafo.text:
            p = paragrafo._element
            p.getparent().remove(p)
            doc.add_paragraph()
            criar_tabela(doc, dados)
            break

In [53]:
def substituir_grafico_no_documento(doc, marcador, buffer):
    for paragrafo in doc.paragraphs:
        if marcador in paragrafo.text:
            p = paragrafo._element
            p.getparent().remove(p)
            doc.add_paragraph()
            doc.add_picture(buffer, width=Inches(6))
            break

In [54]:
def atualizar_documento(template_path, texto, csv_path, output_path):
    # Lê os dados do CSV
    dados = pd.read_csv(csv_path)
    
    # Cria o gráfico
    buffer = criar_grafico(dados)
    
    # Carrega o documento template
    doc = Document(template_path)
    
    # Substitui o texto
    substituir_texto_no_documento(doc, '<TEXTO>', texto)
    
    # Substitui a tabela
    substituir_tabela_no_documento(doc, '<TABELA>', dados)
    
    # Substitui o gráfico
    substituir_grafico_no_documento(doc, '<GRAFICO>', buffer)
    
    # Salva o documento modificado com um novo nome
    doc.save(output_path)

In [55]:
# Parâmetros
template_path = 'template_relatorio.docx'
texto = 'Este é o texto substituto'
csv_path = 'dados_exemplo.csv'
output_path = 'relatorio_atualizado.docx'

# Executa a atualização do documento
atualizar_documento(template_path, texto, csv_path, output_path)